In [1]:
from qibo.cskim_utils.gen_circuit import qibo_to_qiskit_cz, random_control_circuit_qibo
from qibo.cskim_utils.connectivity import *
from qibo.cskim_utils.qiskit_sabre import SabreSwap_old
from qibo.transpiler.optimizer import Preprocessing
from qibo.transpiler.pipeline import Passes
from qibo.transpiler.placer import Trivial
from qibo.transpiler.router import Sabre
from qiskit.transpiler import CouplingMap, Layout, PassManager
from qiskit.transpiler.passes import SetLayout, ApplyLayout
from qiskit.transpiler.passes import SabreSwap
from time import time
import numpy as np
import matplotlib.pyplot as plt
from qibo.cskim_utils.circuit_info import count_swaps_qibo, count_swaps_qiskit


In [5]:
nqubits, ngates = 20, 50

conn = cycle_connectivity_nx(nqubits)
cmap = cycle_connectivity_cmap(nqubits)

cz_circuit_qibo = random_control_circuit_qibo(nqubits, ngates)
cz_circuit_qiskit = qibo_to_qiskit_cz(cz_circuit_qibo)

router_sabre = Sabre(connectivity=conn, swap_threshold=1.5)
placer_trivial = Trivial(connectivity=conn)
pipeline_qibo_sabre = Passes([placer_trivial, router_sabre], connectivity=conn)
coup = CouplingMap(couplinglist=cmap)
q = cz_circuit_qiskit.qubits

pass_qiskit_sabre_old = PassManager()
pass_qiskit_sabre_old.append(SetLayout(Layout({q[i]: i for i in range(nqubits)})))
pass_qiskit_sabre_old.append(ApplyLayout())
qiskit_router_sabre = SabreSwap_old(coup, "decay")
pass_qiskit_sabre_old.append(qiskit_router_sabre)

In [6]:
qibo_sabre_start = time()
transpiled_qibo_sabre, _ = pipeline_qibo_sabre(cz_circuit_qibo)
qibo_sabre_end = time()
swap_count_qibo_sabre = count_swaps_qibo(transpiled_qibo_sabre)
print(qibo_sabre_end - qibo_sabre_start)

qiskit_sabre_old_start = time()
transpiled_qiskit_sabre_old = pass_qiskit_sabre_old.run(cz_circuit_qiskit)
qiskit_sabre_old_end = time()
swap_count_qiskit_sabre_old = count_swaps_qiskit(transpiled_qiskit_sabre_old)
print(qiskit_sabre_old_end - qiskit_sabre_old_start)

Time of preprocessing: 0.0038652420043945312
reset
reset
reset
reset
reset
reset
reset
reset
reset
reset
reset
reset
reset
reset
loop time: [0.2019033432006836, 0.18752741813659668, 0.004248142242431641]
check_execute time: 0.0008380413055419922 0.0015420913696289062 0.00025177001953125 0.19911694526672363
0.5182480812072754
Time preprocessing:  0.00033473968505859375
Loop time:  [0.0007662773132324219, 0.03254365921020508, 0.00029754638671875]
0.03590679168701172
